In [27]:
import os
import tkinter as tk
from tkinter import simpledialog
from selenium import webdriver
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time
from selenium.webdriver.common.action_chains import ActionChains
from dotenv import load_dotenv

# Cargar variables de entorno desde el archivo .env
load_dotenv()

# Función para obtener credenciales desde el .env o solicitar al usuario
def get_credentials():
    email = os.getenv('EMAIL')
    password = os.getenv('PASSWORD')

    if not email or not password:
        root = tk.Tk()
        root.withdraw()  # Oculta la ventana principal
        email = simpledialog.askstring("Credenciales", "Introduce tu correo electrónico:")
        password = simpledialog.askstring("Credenciales", "Introduce tu contraseña:", show='*')

    return email, password

# Función para descargar el informe en PDF
def download_pdf(driver, wait):
    try:
        # Abre el menú de exportación
        export_menu_btn = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, "[data-testid='appbar-export-options-btn'] > span")))
        export_menu_btn.click()

        # Selecciona la opción de exportar a PDF
        export_to_pdf_btn = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="export-to-pdf-btn"]')))
        export_to_pdf_btn.click()

        from selenium.common.exceptions import NoSuchElementException

        try:
            # Intentar encontrar el checkbox usando el atributo name
            exclude_hidden_tabs_checkbox = driver.find_element(By.NAME, "exclude-hidden-slides")
            print("[INFO] Elemento 'exclude-hidden-slides' encontrado.")

            # Verificar si está seleccionado
            if exclude_hidden_tabs_checkbox.is_selected():
                exclude_hidden_tabs_checkbox.click()  # Hacemos clic si está seleccionado
                print("[INFO] Checkbox desmarcado.")
            else:
                print("[INFO] Checkbox ya estaba desmarcado.")

        except NoSuchElementException:
            print("[ERROR] Elemento 'exclude-hidden-slides' no encontrado.")

        # Repetir para el segundo checkbox
        try:
            # Intentar encontrar el checkbox "Exportar solo la página actual"
            export_current_page_checkbox = WebDriverWait(driver, 30).until(
                EC.element_to_be_clickable((By.NAME, "export-only-current-page"))
            )
            print("[INFO] Elemento 'export-only-current-page' encontrado.")

            # Verificar si está seleccionado
            if export_current_page_checkbox.is_selected():
                export_current_page_checkbox.click()  # Hacemos clic si está seleccionado
                print("[INFO] Checkbox 'export-only-current-page' desmarcado.")
            else:
                print("[INFO] Checkbox 'export-only-current-page' ya estaba desmarcado.")

        except NoSuchElementException:
            print("[ERROR] Elemento 'export-only-current-page' no encontrado.")
        except Exception as e:
            print(f"[ERROR] Ocurrió un error al manejar 'export-only-current-page': {e}")
            
        # Confirma la descarga
        ok_button = wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@data-testid="dailog-ok-btn"]')))
        ok_button.click()

        print("Informe PDF descargándose...")

        # Verifica si el archivo se ha descargado
        current_directory = os.getcwd()
        filename = "nombre_del_archivo.pdf"  # Cambia esto al nombre esperado del archivo PDF
        if is_file_downloaded(current_directory, filename):
            print("La descarga del PDF se completó con éxito.")
        else:
            print("La descarga del PDF no se completó en el tiempo esperado.")

    except Exception as e:
        print(f"Error al descargar el PDF: {e}")

# Función para verificar si el archivo se ha descargado
def is_file_downloaded(directory, filename, timeout=60):
    start_time = time.time()
    file_path = os.path.join(directory, filename)
    
    while time.time() - start_time < timeout:
        if os.path.isfile(file_path) and os.path.getsize(file_path) > 0:
            # Verifica si el archivo está siendo descargado (tamaño cambia)
            initial_size = os.path.getsize(file_path)
            time.sleep(5)  # Espera unos segundos
            final_size = os.path.getsize(file_path)
            if initial_size == final_size:
                return True
    return False

# Función para enviar claves con reintentos
def send_keys_with_retry(element, keys, retries=3):
    for _ in range(retries):
        try:
            element.send_keys(keys)
            return
        except Exception as e:
            print(f"Error al enviar las teclas: {e}")
            time.sleep(1)  # Espera un segundo y vuelve a intentar
    raise Exception("No se pudo enviar las teclas al elemento después de varios intentos.")

# Configura las opciones de Firefox
options = Options()
# options.add_argument("--headless")  # Ejecutar en modo headless si no quieres ver la UI
options.set_preference('general.useragent.override', 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Firefox/74.0')
options.set_preference('network.proxy.type', 1)  # Configura el uso del proxy
options.set_preference('network.proxy.http', 'localhost')
options.set_preference('network.proxy.http_port', 8080)
options.set_preference('network.proxy.ssl', 'localhost')
options.set_preference('network.proxy.ssl_port', 8080)
options.set_preference('security.enterprise_roots.enabled', True)  # Ignorar los errores de certificados

# Usa el directorio actual como ruta de descarga
current_directory = os.getcwd()
options.set_preference("browser.download.folderList", 2)  # Permitir descarga en una ubicación específica
options.set_preference("browser.download.dir", current_directory)  # Establece el directorio actual como ruta de descarga
options.set_preference("browser.helperApps.neverAsk.saveToDisk", "application/pdf")

# Inicia el navegador
driver = webdriver.Firefox(options=options)
wait = WebDriverWait(driver, 30)

# Abre la página de Power BI
driver.get("https://app.powerbi.com/groups/9cb47af7-81b9-490c-bf55-6533762e5cca/reports/47bb2d84-fe3b-4e50-8ce4-c2df1ea1e500/ReportSection4291253ac18667144247?experience=power-bi")

try:
    # Intenta detectar el inicio de sesión
    email_field = wait.until(EC.visibility_of_element_located((By.ID, 'email')))
    if email_field:
        # Obtiene las credenciales
        email, password = get_credentials()
        send_keys_with_retry(email_field, email)
        email_field.send_keys(Keys.RETURN)

        # Espera a que aparezca el campo de contraseña
        password_field = wait.until(EC.visibility_of_element_located((By.CSS_SELECTOR, 'input[type="password"]')))
        send_keys_with_retry(password_field, password)
        password_field.send_keys(Keys.RETURN)

        # Detecta la ventana de "¿Quiere mantener la sesión iniciada?"
        keep_signed_in = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'input[id="idBtn_Back"]')))
        keep_signed_in.click()  # Hace clic en "No"
        download_pdf(driver, wait)

except Exception as e:
    print(f"Ocurrió un error: {e}")

finally:
    # Cierra el navegador
    driver.quit()


[INFO] Elemento 'exclude-hidden-slides' encontrado.
[INFO] Checkbox ya estaba desmarcado.
[INFO] Elemento 'export-only-current-page' encontrado.
[INFO] Checkbox 'export-only-current-page' ya estaba desmarcado.
Informe PDF descargándose...
La descarga del PDF no se completó en el tiempo esperado.


In [ ]:
# mitmdump -s mitm_scr_2.py "~h X-PowerBI"